# ILP Solver Notebook

## Solver Implementations

### Utils

In [1]:
from typing import List
import pandas as pd

# Convert max throughput profiling to a mapping from request size to load
def tputs_to_loads_2d(max_tputs: List[List[float]]):
    loads = []
    for i in range(len(max_tputs)):
        loads.append([])
        for j in range(len(max_tputs[0])):
            load = 1 / max_tputs[i][j]
            loads[-1].append(load)
    return loads

def display_experiment_results(results, solver_labels, ilp_result):
  df = pd.DataFrame(results)
  df.fillna(0, inplace=True)

  # Add the last column filled with zeros
  df['Savings'] = [str(round(((x["cost"] - ilp_result["cost"])/x["cost"]) * 100, 2)) + "%" for x in results]

  # Ensure the 'cost' column is second to last and 'LastColumn' is last, this step might need adjustment based on actual GPU types
  # Assuming we don't know all GPU types in advance, let's dynamically sort columns except for 'cost' and 'LastColumn'
  gpu_columns = [col for col in df.columns if col not in ['cost', 'Savings']]
  sorted_columns = gpu_columns + ['cost', 'Savings']
  df = df[sorted_columns]
  df.index = solver_labels

  # Display the table
  print(df)

### ILP Solver for Heterogeneous Accelerators

In [3]:
import pulp
from pulp import LpVariable, LpProblem, LpMinimize, LpInteger

def run_ILP_solver(workload_distribution, overall_rate, slice_factor, gpu_info, logs=False):
    # Multiply overall rate across distribution.
    request_rate_histogram = []
    for i in range(len(workload_distribution)):
        request_rate_histogram.append([])
        for j in range(len(workload_distribution[0])):
            request_rate_histogram[-1].append(workload_distribution[i][j] * overall_rate)

    # Convert the profiled max throughputs into mapping from request size to load
    for gpu in gpu_info:
        gpu_info[gpu]["loads"] = tputs_to_loads_2d(gpu_info[gpu]["tputs"])

    gpu_types = list(gpu_info.keys())
    cost_vector = [gpu_info[gpu]["cost"] for gpu in gpu_types]

    # Create slices, which is a single dimension.
    slices = []
    for i in range(len(request_rate_histogram)):
      for j in range(len(request_rate_histogram[i])):
        for _ in range(slice_factor):
            slices.append(request_rate_histogram[i][j] / slice_factor)

    # Create slice-to-load mapping, which is a single dimension.
    for gpu in gpu_types:
        slice_loads = []
        for i in range(len(gpu_info[gpu]["loads"])):
            for j in range(len(gpu_info[gpu]["loads"][i])):
                for _ in range(slice_factor):
                    slice_loads.append(gpu_info[gpu]["loads"][i][j])
        assert len(slices) == len(slice_loads)
        gpu_info[gpu]["slice_loads"] = slice_loads


    # Decision matrix value is binary. The slice is assigned to a GPU, or it isn't.
    matrix_rows = len(slices)
    matrix_cols = len(gpu_types)

    # Vector value is non-negative integer of how many of each GPU type are needed
    vector_length = matrix_cols

    decision_matrix = [[LpVariable(f"x_{i}_{j}", cat=LpInteger, lowBound=0, upBound=1) for j in range(matrix_cols)] for i in range(matrix_rows)]
    decision_vector = [LpVariable(f"y_{i}", cat=LpInteger, lowBound=0) for i in range(vector_length)]

    # Objective: minimize cost
    problem = LpProblem("GpuAllocation", LpMinimize)
    problem += pulp.lpSum([decision_vector[i] * cost_vector[i] for i in range(len(decision_vector))])

    # C1: Each row of decision matrix must sum to exactly 1 (ie, each slice assigned to one GPU)
    for i in range(len(decision_matrix)):
        problem += pulp.lpSum(decision_matrix[i]) == 1

    # C2: Load of column of decision matrix must fit in decision vector capacity
    for j in range(len(decision_matrix[0])):
        # j is idx of GPU type, i is slice
        problem += pulp.lpSum([decision_matrix[i][j] * gpu_info[gpu_types[j]]["slice_loads"][i] * slices[i] for i in range(len(decision_matrix))]) <= decision_vector[j]

    # Solve the problem
    problem.solve(pulp.PULP_CBC_CMD(msg=0))

    # Print the results
    if logs:
        print(f'Decision Matrix:')
        for row in decision_matrix:
            print([var.value() for var in row])
        print(f'Decision Vector:')
        print(f'{[var.value() for var in decision_vector]}')

    if pulp.LpStatus[problem.status] != 'Optimal':
        return None

    solution_dict = {}
    for i in range(len(decision_vector)):
        solution_dict[gpu_types[i]] = decision_vector[i].value()

    total_cost = 0
    for gpu in solution_dict:
        total_cost += solution_dict[gpu] * gpu_info[gpu]["cost"]
    solution_dict["cost"] = total_cost

    return solution_dict


## Running the Solver
Choose the parameters of the experiment setting.

In [5]:
# Example inputs, replace them with your own data
gpu_info_example = {
   "A10G" : {
      "cost": 1.01,
      "tputs": [[5, 1],
                [10,5]],
   },
   "A100" : {
      "cost": 3.67,
      "tputs": [[20, 2],
                [50, 20]],
   }
}
workload_distribution = [[0.25, 0.5],
                         [0.25, 0.25]]
overall_rate = 16
slice_factor = 1

# Run the ILP solver
print(run_ILP_solver(workload_distribution=workload_distribution, overall_rate=overall_rate, slice_factor=slice_factor, gpu_info=gpu_info_example, logs=False))

{'A10G': 10.0, 'A100': 0.0, 'cost': 10.1}
